In [1]:
import os
import pandas as pd
import json
base_path = '/hpc2hdd/home/ypeng455/聊天记录'
suffix = '_utf8.csv'
chatnames = os.listdir(base_path)
for chatname in chatnames:
    image_path = base_path + chatname + '/image/'
    csv_path = base_path + chatname +'/'+ chatname + suffix
    try:
        pd_csv = pd.read_csv(csv_path)
    except:
        continue
    df = pd_csv.loc[(pd_csv['Type']==1) | (pd_csv['Type']==3)]
    df = df.reset_index(drop=True)

    remain_tokens = 2048
    time_threshold = 3600
    photo_tokens = 256
    template_tokens = 30
    sentence = ''
    picture_num = 1
    conversations = []
    group = False
    if len(df['Sender'].unique()) > 2:
        group = True
    for index, row in df.iterrows():
        # 检查是否为最后一行，避免索引越界
        if index < len(df) - 1:
            if conversations == []:#添加system_input
                if group:
                    system = {
                        "from":"system",
                        "value":f"你是彭一洋,微信昵称为Antigone,大家会称呼你为pyy,彭总，彭老师，下面你将要在群聊{chatname}中进行群聊对话"
                    }
                else:
                    try:
                        id_row = df[df['NickName'] != 'Antigone'].iloc[0]
                    except:
                        continue
                    system = {
                        "from":"system",
                        "value":f"你是彭一洋,微信昵称为Antigone,大家会称呼你为pyy,彭总，彭老师，下面你将要与{','.join([id_row['Remark'],id_row['NickName'],id_row['Sender']])}进行一对一对话"
                    }
                conversations.append(system)
            #判断要不要开启新的conversation
            next_input = df.iloc[index + 1]
            if remain_tokens > 0 and next_input['CreateTime'] - row['CreateTime'] < time_threshold:
                if len(conversations) == 1 and row['IsSender']:
                    continue    #跳过非对方发起的对话，因为model不该主动发起对话
                
                # 考虑要不要合并sentence
                if  next_input['Sender'] == row['Sender']:
                    if row['Type'] == 1:
                        sentence += row['StrContent'] + '/n'
                        remain_tokens -= len(row['StrContent'])
                    elif row['Type'] == 3:
                        sentence += f"Picture {picture_num}: <img>{row['CreateTime']}.jpg</img>\n"
                        picture_num+=1
                        remain_tokens -= photo_tokens
                else:
                    if row['Type'] == 1:
                        sentence += row['StrContent'] + '/n'
                        remain_tokens -= len(row['StrContent'])
                    elif row['Type'] == 3:
                        sentence += f"Picture {picture_num}: <img>{row['CreateTime']}.jpg</img>\n"
                        picture_num+=1
                        remain_tokens -= photo_tokens
                    role = 'user' if not row['IsSender'] else 'assistant'
                    unit = {"from":role,
                            "value":sentence+"来自:"+','.join([row['Remark'],row['NickName'],row['Sender']])}
                    conversations.append(unit)
                    sentence = ''
                    
            else:
                if 'user' in str(conversations) and 'assistant' in str(conversations):
                    new_data = {"id":"",
                                "conversations":conversations}
                    json_file_path = 'path_to_your_json_file.json'

                    # 读取现有的JSON文件内容到列表对象
                    with open(json_file_path, 'r', encoding='utf-8') as f:
                        data = json.load(f)
                    if data == '':
                        data = []
                    # 将新数据字典添加到列表的末尾
                    data.append(new_data)

                    # 将更新后的列表写回到JSON文件
                    with open(json_file_path, 'w', encoding='utf-8') as f:
                        json.dump(data, f, ensure_ascii=False, indent=4)
                
                remain_tokens,picture_num,conversations = 2048,1,[]
        else:
            # 这是最后一行，没有下一行可以处理
            print("这是最后一行数据。",index)
            break

In [2]:
!pwd

/hpc2hdd/home/ypeng455/Qwen-VL
